<a href="https://colab.research.google.com/github/tigerofmurder/bioinformatica/blob/main/bio_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np


df = pd.DataFrame([[2,-7,-5,-7],[-7,2,-7,-5],[-5,-7,2,-7],[-7,-5,-7,2]], index=["A", "C", "G", "T"], columns=["A", "C", "G", "T"])

df.head()

,A,C,G,T
A,2,-7,-5,-7
C,-7,2,-7,-5
G,-5,-7,2,-7
T,-7,-5,-7,2


### **Funcion para crear la matriz de miss_match y match**

In [3]:
def make_match_missmatch_score(A,B,missmatch,match):
  values = set(A+B)
  an_array = np.full((len(values), len(values)), missmatch)
  np.fill_diagonal(an_array,match)
  df = pd.DataFrame(an_array, index=values, columns=values)
  return df

### **Alineamiento de secuencias**

In [55]:
import itertools
def cond_diagonal(str1,str2,A,B,d,s,i,j,matrix,previus_state=True):
  if matrix[j][i] == matrix[j-1][i-1] + s[A[i]][B[j]] or previus_state==False:
    str1+=A[i]
    str2+=B[j]
    i,j = i-1,j-1
    return True,str1,str2,i,j
  else:
    return False,str1,str2,i,j
def cond_up(str1,str2,A,B,d,s,i,j,matrix,previus_state=True):
  if matrix[j][i] == matrix[j-1][i]+d or previus_state==False:
    str1+="-"
    str2+=B[j]
    j = j-1
    return True,str1,str2,i,j
  else:
    return False,str1,str2,i,j
    
def cond_back(str1,str2,A,B,d,s,i,j,matrix,previus_state=True):
  if matrix[j][i] == matrix[j][i-1]+d or previus_state==False:
    str1+=A[i]
    str2+="-"
    i = i-1
    return True,str1,str2,i,j
  else:
    return False,str1,str2,i,j


def full_order(A,B,matrix,s,d):
  matrix_values = []
  array = [cond_diagonal,cond_back,cond_up]
  for subset in itertools.permutations(array, 3):
    #print(subset)
    str1 = ""
    str2 = ""
    i,j = len(A)-1,len(B)-1
    while i>0 and j>0:
      if subset[0](str1,str2,A,B,d,s,i,j,matrix)[0]:
        bool_,str1,str2,i,j = subset[0](str1,str2,A,B,d,s,i,j,matrix)
      elif subset[1](str1,str2,A,B,d,s,i,j,matrix)[0]:
        bool_,str1,str2,i,j = subset[1](str1,str2,A,B,d,s,i,j,matrix)
      else:
        bool_,str1,str2,i,j = subset[2](str1,str2,A,B,d,s,i,j,matrix)
    while i>0:
      str1+=A[i]
      str2+="-"
      i = i-1
    while j>0:
      str1+="-"
      str2+=B[j]
      j = j-1
    cont_sum = 0
    for i in range(len(str1)):
      if str1[i]==str2[i]:
        cont_sum+=s[str1[i]][str2[i]]
      elif str1[i]=="-" or str2[i]=="-":
        cont_sum+=d
      else:
        cont_sum+=s[str1[i]][str2[i]]
    matrix_values.append([str1[::-1],str2[::-1],cont_sum])
  return matrix_values

# d es gap
def funcion(A,B,d,s=df,missmatch = -2,match = 2,m_b = False):
  if m_b==True:
    s = make_match_missmatch_score(A,B,missmatch,match)
  A = "-"+A
  A = [i for i in A]
  B = "-"+B
  B = [i for i in B]
  print(s.head())
  matrix = np.zeros((len(B),len(A)))
  
  matrix[0,:] = [d*i for i in range(len(A))]
  matrix[:,0] = [d*i for i in range(len(B))]

  for i in range(1,len(A)):
    for j in range(1,len(B)):
        v1 = matrix[j-1][i-1] + s[A[i]][B[j]]
        v3 = matrix[j-1][i] + d
        v2 = matrix[j][i-1] + d
        matrix[j][i] = max(v1,v2,v3)
  matrix_values = full_order(A,B,matrix,s,d)
  Values_DF = pd.DataFrame(np.transpose(np.array(matrix_values)),index=["sequ1","sequ2","score"])
  print(Values_DF)
  print(Values_DF.T.drop_duplicates().T)
  return matrix,A,B

values,A,B = funcion("AGC","AAG",d=-5,m_b=False)

pdyn = pd.DataFrame(values, index=B, columns=A)
pdyn

   A  C  G  T
A  2 -7 -5 -7
C -7  2 -7 -5
G -5 -7  2 -7
T -7 -5 -7  2
          0     1     2     3     4     5
sequ1  -AGC  -AGC  -AGC  A-GC  A-GC  A-GC
sequ2  AAG-  AAG-  AAG-  AAG-  AAG-  AAG-
score    -6    -6    -6    -6    -6    -6
          0     3
sequ1  -AGC  A-GC
sequ2  AAG-  AAG-
score    -6    -6


,-,A,G,C
-,0.0,-5.0,-10.0,-15.0
A,-5.0,2.0,-3.0,-8.0
A,-10.0,-3.0,-3.0,-8.0
G,-15.0,-8.0,-1.0,-6.0


In [8]:
def order1(A,B,matrix,s,d):
  str1 = ""
  str2 = ""
  i,j = len(A)-1,len(B)-1
  while i>0 and j>0:
    if matrix[j][i] == matrix[j-1][i]+d:
      str1+="-"
      str2+=B[j]
      j = j-1
      #print("Second com")
    elif matrix[j][i] == matrix[j-1][i-1] + s[A[i]][B[j]]:
      str1+=A[i]
      str2+=B[j]
      i,j = i-1,j-1
      #print("First com")
    else:
      str1+=A[i]
      str2+="-"
      i = i-1
      #print("3 com")
  while i>0:
    str1+=A[i]
    str2+="-"
    i = i-1
  while j>0:
    str1+="-"
    str2+=B[j]
    j = j-1
  print("Secuencia 1: ",str1[::-1])
  print("Secuencia 2: ",str2[::-1])
  return str1,str2


def order2(A,B,matrix,s,d):
  str1 = ""
  str2 = ""
  i,j = len(A)-1,len(B)-1
  while i>0 and j>0:
    if matrix[j][i] == matrix[j-1][i-1] + s[A[i]][B[j]]:
      str1+=A[i]
      str2+=B[j]
      i,j = i-1,j-1
      #print("First com")
    elif matrix[j][i] == matrix[j-1][i]+d:
      str1+="-"
      str2+=B[j]
      j = j-1
      #print("Second com")
    else:
      str1+=A[i]
      str2+="-"
      i = i-1
      #print("3 com")
  while i>0:
    str1+=A[i]
    str2+="-"
    i = i-1
  while j>0:
    str1+="-"
    str2+=B[j]
    j = j-1
  print("Secuencia 1: ",str1[::-1])
  print("Secuencia 2: ",str2[::-1])
  return str1,str2


def order3(A,B,matrix,s,d):
  str1 = ""
  str2 = ""
  i,j = len(A)-1,len(B)-1
  while i>0 and j>0:
    if matrix[j][i] == matrix[j][i-1]+d:
      str1+=A[i]
      str2+="-"
      i = i-1
      #print("3 com")
    elif matrix[j][i] == matrix[j-1][i-1] + s[A[i]][B[j]]:
      str1+=A[i]
      str2+=B[j]
      i,j = i-1,j-1
      #print("First com")
    else:
      str1+="-"
      str2+=B[j]
      j = j-1
      #print("Second com")
  while i>0:
    str1+=A[i]
    str2+="-"
    i = i-1
  while j>0:
    str1+="-"
    str2+=B[j]
    j = j-1
  print("Secuencia 1: ",str1[::-1])
  print("Secuencia 2: ",str2[::-1])
  return str1,str2

In [9]:
def funcion(A,B,d,s=df,missmatch = -2,match = 2,m_b = False,order_fun=order1):
  if m_b==True:
    s = make_match_missmatch_score(A,B,missmatch,match)
  A = "-"+A
  A = [i for i in A]
  B = "-"+B
  B = [i for i in B]
  print(s.head())
  matrix = np.zeros((len(B),len(A)))
  
  matrix[0,:] = [d*i for i in range(len(A))]
  matrix[:,0] = [d*i for i in range(len(B))]

  for i in range(1,len(A)):
    for j in range(1,len(B)):
        v1 = matrix[j-1][i-1] + s[A[i]][B[j]]
        v3 = matrix[j-1][i] + d
        v2 = matrix[j][i-1] + d
        matrix[j][i] = max(v1,v2,v3)
  str1, str2 = order_fun(A,B,matrix,s,d)
  cont_sum = 0
  for i in range(len(str1)):
    if str1[i]==str2[i]:
      cont_sum+=s[str1[i]][str2[i]]
    elif str1[i]=="-" or str2[i]=="-":
      cont_sum+=d
    else:
      cont_sum+=s[str1[i]][str2[i]]
  print("Score: ",cont_sum)
  return matrix,A,B
#values,A,B = funcion("AGC","AAAC",d=-2,missmatch=-1,match=1,m_b=True)
#values,A,B = funcion("AGC","AAAC",d=-2,s=df)
#values,A,B = funcion("TTCG","ATAG",d=-1,missmatch=-1,match=1,m_b=True)
#values,A,B = funcion("TTCG","ATAG",d=-2)
values,A,B = funcion("AGC","AAG",d=-5,m_b=False,order_fun = order1)
values,A,B = funcion("AGC","AAG",d=-5,m_b=False,order_fun = order2)
values,A,B = funcion("AGC","AAG",d=-5,m_b=False,order_fun = order3)
#values,A,B = funcion("ATACTGGG","TGACTGAG",d=-2)

pdyn = pd.DataFrame(values, index=B, columns=A)
pdyn

   A  C  G  T
A  2 -7 -5 -7
C -7  2 -7 -5
G -5 -7  2 -7
T -7 -5 -7  2
Secuencia 1:  A-GC
Secuencia 2:  AAG-
Score:  -6
   A  C  G  T
A  2 -7 -5 -7
C -7  2 -7 -5
G -5 -7  2 -7
T -7 -5 -7  2
Secuencia 1:  -AGC
Secuencia 2:  AAG-
Score:  -6
   A  C  G  T
A  2 -7 -5 -7
C -7  2 -7 -5
G -5 -7  2 -7
T -7 -5 -7  2
Secuencia 1:  -AGC
Secuencia 2:  AAG-
Score:  -6


,-,A,G,C
-,0.0,-5.0,-10.0,-15.0
A,-5.0,2.0,-3.0,-8.0
A,-10.0,-3.0,-3.0,-8.0
G,-15.0,-8.0,-1.0,-6.0
